In [3]:
import os
import pandas as pd
import dask.dataframe as dkdd
import warnings
warnings.filterwarnings(action="ignore")

In [4]:
os.environ['KAGGLE_USERNAME'] = " "
os.environ['KAGGLE_KEY'] = " "

In [5]:
!kaggle datasets download -d arevel/chess-games

chess-games.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
from zipfile import ZipFile
file_name = "/content/chess-games.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [7]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string
def col_header_val(df, table_config):
    cols = df.columns
    cols = cols.str.strip()
    cols.str.replace(" ", "_")
    expected_col = list(map(lambda x: x.lower(),  config_data['columns']))
    cols = list(cols.sort_values())
    expected_col.sort()
    if len(cols) == len(expected_col) and cols == expected_col:
      print("Column name and Column length Validation Passed!!")
      return 1
    else:
      print("Column name and Column length Validation Failed..")
      mismatched_columns_file = list(set(cols).difference(expected_col))
      print("Following File columns are not in the YAML file", mismatched_columns_file)
      missing_YAML_file = list(set(expected_col).difference(cols))
      print("Following YAML columns are not in the file uploaded", missing_YAML_file)
      logging.info(f'df columns: {cols}')
      logging.info(f'expected columns: {expected_col}')
      return 0
def humanbytes(B):
   'Return the given bytes as a human friendly KB, MB, GB, or TB string'
   B = float(B)
   KB = float(1024)
   MB = float(KB ** 2) # 1,048,576
   GB = float(KB ** 3) # 1,073,741,824
   TB = float(KB ** 4) # 1,099,511,627,776

   if B < KB:
      return '{0} {1}'.format(B,'Bytes' if 0 == B > 1 else 'Byte')
   elif KB <= B < MB:
      return '{0:.2f} KB'.format(B/KB)
   elif MB <= B < GB:
      return '{0:.2f} MB'.format(B/MB)
   elif GB <= B < TB:
      return '{0:.2f} GB'.format(B/GB)
   elif TB <= B:
      return '{0:.2f} TB'.format(B/TB)

def stats(df, config_data):
  if col_header_val(df, config_data) == 1:
    col_names = list(df.columns)
    no_of_cols = df.shape[1]
    no_of_rows = df.shape[0]
    size = df.memory_usage(deep=True).sum()
    file_size = humanbytes(size)
    statistics = f"\nNo. of Columns: {no_of_cols} \nNo. of Rows: {no_of_rows} \nColumn Names: {col_names} \nFile Size: {file_size}"
    print(statistics)

def save_gz(df):
  df.to_csv('compressed_train.gz', sep='|', compression='gzip')
  print("File saved in gz format with pipe separator")

Overwriting testutility.py


In [8]:
%%writefile file.yaml
file_type: csv
dataset_name: ChessGames
file_name: chess-games
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - Event
    - White
    - Black
    - Result
    - UTCDate
    - UTCTime
    - WhiteElo
    - BlackElo
    - WhiteRatingDiff
    - BlackRatingDiff
    - ECO
    - Opening
    - TimeControl
    - Termination
    - AN

Overwriting file.yaml


In [9]:
import testutility as util
config_data = util.read_config_file("file.yaml")

In [10]:
config_data['inbound_delimiter']

','

In [11]:

config_data

{'columns': ['Event',
  'White',
  'Black',
  'Result',
  'UTCDate',
  'UTCTime',
  'WhiteElo',
  'BlackElo',
  'WhiteRatingDiff',
  'BlackRatingDiff',
  'ECO',
  'Opening',
  'TimeControl',
  'Termination',
  'AN'],
 'dataset_name': 'ChessGames',
 'file_name': 'chess-games',
 'file_type': 'csv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'table_name': 'edsurv'}

Using Pandas

In [16]:
%%time
df = pd.read_csv('chess_games.csv', sep = config_data['inbound_delimiter'])
print(f'Number of Rows: {len(df)}, Number of Columns: {len(df.columns)}')
print(f"Size: {os.path.getsize('chess_games.csv')/10**9:.2f} GB")

Number of Rows: 6256184, Number of Columns: 15
Size: 4.38 GB
CPU times: user 1min 5s, sys: 6.77 s, total: 1min 12s
Wall time: 1min 18s


Using Dask

In [17]:
%%time
df = dkdd.read_csv('chess_games.csv')
print(f'Number of Rows: {len(df)}, Number of Columns: {len(df.columns)}')
print(f"Size: {os.path.getsize('chess_games.csv')/10**9:.2f} GB")

Number of Rows: 6256184, Number of Columns: 15
Size: 4.38 GB
CPU times: user 1min 25s, sys: 6.1 s, total: 1min 31s
Wall time: 1min 13s


In [18]:
df.columns

Index(['Event', 'White', 'Black', 'Result', 'UTCDate', 'UTCTime', 'WhiteElo',
       'BlackElo', 'WhiteRatingDiff', 'BlackRatingDiff', 'ECO', 'Opening',
       'TimeControl', 'Termination', 'AN'],
      dtype='object')

In [20]:
import datetime
import csv
import gzip
df.to_csv('chess_gamess.gz',
          sep='|',
          header=True,
          index=False,
          quoting=csv.QUOTE_ALL,
          compression='gzip',
          quotechar='"',
          doublequote=True,
          line_terminator='\n')

['/content/chess_gamess.gz/00.part',
 '/content/chess_gamess.gz/01.part',
 '/content/chess_gamess.gz/02.part',
 '/content/chess_gamess.gz/03.part',
 '/content/chess_gamess.gz/04.part',
 '/content/chess_gamess.gz/05.part',
 '/content/chess_gamess.gz/06.part',
 '/content/chess_gamess.gz/07.part',
 '/content/chess_gamess.gz/08.part',
 '/content/chess_gamess.gz/09.part',
 '/content/chess_gamess.gz/10.part',
 '/content/chess_gamess.gz/11.part',
 '/content/chess_gamess.gz/12.part',
 '/content/chess_gamess.gz/13.part',
 '/content/chess_gamess.gz/14.part',
 '/content/chess_gamess.gz/15.part',
 '/content/chess_gamess.gz/16.part',
 '/content/chess_gamess.gz/17.part',
 '/content/chess_gamess.gz/18.part',
 '/content/chess_gamess.gz/19.part',
 '/content/chess_gamess.gz/20.part',
 '/content/chess_gamess.gz/21.part',
 '/content/chess_gamess.gz/22.part',
 '/content/chess_gamess.gz/23.part',
 '/content/chess_gamess.gz/24.part',
 '/content/chess_gamess.gz/25.part',
 '/content/chess_gamess.gz/26.part',
 